# Análisis exploratorio y limpieza de datos

In [155]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import glob, os
import datetime

## Importando los csv's

In [12]:
## Importando los datos para PM10
path = r'C:\Users\luisf\Documents\Python Projects\datamex1019\air-quality-cdmx\data'                     # use your path
all_files = glob.glob(os.path.join(path, "*PM10.xls"))     # advisable to use os.path.join as this makes concatenation OS independent
all_files

['C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2000PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2001PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2002PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2003PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2004PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2005PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2006PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2007PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2008PM10.xls',
 'C:\\Users\\luisf\\Documents\\Python Projects\\datamex1019\\air-quality-cdmx\\data\\2009PM10.xls',


In [68]:
df_from_each_file = (pd.read_excel(f) for f in all_files)
df_from_each_file
concatenated_df=pd.concat([i.set_index(['FECHA']+['HORA']) for i in df_from_each_file]).reset_index()

C:\Users\luisf\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [72]:
concatenated_df.shape

(173856, 37)

In [70]:
concatenated_df.columns

Index(['FECHA', 'HORA', 'ACO', 'AJM', 'ATI', 'BJU', 'CAM', 'CES', 'CHO', 'CUA',
       'CUT', 'FAC', 'FAR', 'GAM', 'HAN', 'HGM', 'INN', 'IZT', 'LLA', 'LPR',
       'LVI', 'MER', 'MGH', 'MPA', 'PED', 'PLA', 'SAC', 'SAG', 'SFE', 'SUR',
       'TAH', 'TAX', 'TLA', 'TLI', 'UIZ', 'VIF', 'XAL'],
      dtype='object')

In [75]:
#Filtrando por las alcaldías dentro del alcance
df=concatenated_df[["FECHA","HORA","BJU","HGM","MGH"]]

In [79]:
df.dtypes 

FECHA    datetime64[ns]
HORA              int64
BJU             float64
HGM             float64
MGH             float64
dtype: object

¿Cómo le haré para manejar los NaN's y los valores -99.0 (que son lo mismo)?
- Primero, dividiré el data set por cada estación

In [88]:
#df[["FECHA","HORA","BJU","HGM","MGH"]]
df_benito=df[["FECHA","HORA","BJU"]]
df_cuauh=df[["FECHA","HORA","HGM"]]
df_miguel=df[["FECHA","HORA","MGH"]]

## Limpieza para Benito Juárez

In [126]:
nulls_benito = {}
for i in range(2000,2020):
    nulls_benito[i]=(df_benito[(df_benito.FECHA >= '{}-01-01' .format(i)) & (df_benito.FECHA <= '{}-12-31' .format(i))].isnull().sum()[2])

In [127]:
nulls_benito #Esto quiere decir que para los primeros 14 años de este milenio, no se midió nada en esta estación

{2000: 8784,
 2001: 8760,
 2002: 8760,
 2003: 8760,
 2004: 8784,
 2005: 8760,
 2006: 8760,
 2007: 8760,
 2008: 8784,
 2009: 8760,
 2010: 8760,
 2011: 8760,
 2012: 8784,
 2013: 8760,
 2014: 8760,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0}

In [135]:
df_benito_v2=df_benito[df_benito.FECHA >= '2015-01-01']
df_benito_v2.shape

(42360, 3)

In [136]:
df_benito_v2.BJU.value_counts().head() # 10 mil putos -99.0 

-99.0    10581
 25.0      707
 24.0      679
 23.0      673
 26.0      670
Name: BJU, dtype: int64

In [288]:
df_benito_v2['AÑO']=df_benito_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_benito_v2['MES']=df_benito_v2.FECHA.apply(lambda x: str(x).split('-')[1])

In [289]:
df_benito_v2.head()

,FECHA,HORA,BJU,AÑO,MES
131496,2015-01-01,1,-99.0,2015,01
131497,2015-01-01,2,-99.0,2015,01
131498,2015-01-01,3,-99.0,2015,01
131499,2015-01-01,4,-99.0,2015,01
131500,2015-01-01,5,-99.0,2015,01


Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [290]:
df_benito_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2015,01,744
1,2015,02,672
2,2015,03,744
3,2015,04,720
4,2015,05,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

In [291]:
df_benito_v2[df_benito_v2.BJU == -99.0].groupby(["AÑO","MES"])['BJU'].value_counts().to_frame()

BJU
AÑO  MES BJU       
2015 01  -99.0  744
     02  -99.0  672
     03  -99.0  744
     04  -99.0  720
     05  -99.0  744
     06  -99.0  720
     07  -99.0  744
     08  -99.0  113
     09  -99.0   52
     10  -99.0   46
     11  -99.0    8
     12  -99.0   33
2016 01  -99.0   32
     02  -99.0    4
     03  -99.0   13
     04  -99.0   11
     05  -99.0    3
     06  -99.0    4
     07  -99.0    9
     08  -99.0    3
     09  -99.0   12
     10  -99.0    4
     11  -99.0    7
     12  -99.0   85
2017 01  -99.0   23
     02  -99.0   12
     03  -99.0    7
     04  -99.0   40
     05  -99.0  107
     06  -99.0   22
     07  -99.0    6
     08  -99.0   90
     09  -99.0  100
     10  -99.0   20
     11  -99.0    6
2018 01  -99.0   32
     02  -99.0    5
     03  -99.0    3
     04  -99.0    5
     05  -99.0    5
     06  -99.0   29
     07  -99.0   18
     08  -99.0   14
     09  -99.0   14
     10  -99.0   84
     11  -99.0   56
     12  -99.0   68
2019 01  -99.0  528
     02  -99.0  672
     03  -99.0  744
     04  -99.0  720
     05  -99.0  744
     06  -99.0  102
     07  -99.0  243
     08  -99.0    9
     09  -99.0   17
     10  -99.0  509

## Limpieza para Cuahtémoc

### Encontrar NaN's y -99.0

In [123]:
nulls_cuauh = {}
for i in range(2000,2020):
    nulls_cuauh[i]=(df_cuauh[(df_cuauh.FECHA >= '{}-01-01' .format(i)) & (df_cuauh.FECHA <= '{}-12-31' .format(i))].isnull().sum()[2])

In [124]:
nulls_cuauh #Medición a partir del 2012

{2000: 8784,
 2001: 8760,
 2002: 8760,
 2003: 8760,
 2004: 8784,
 2005: 8760,
 2006: 8760,
 2007: 8760,
 2008: 8784,
 2009: 8760,
 2010: 8760,
 2011: 8760,
 2012: 0,
 2013: 0,
 2014: 0,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0}

In [137]:
df_cuauh_v2=df_cuauh[df_cuauh.FECHA >= '2012-01-01']
df_cuauh_v2.shape

(68664, 3)

In [280]:
df_cuauh_v2['AÑO']=df_cuauh_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_cuauh_v2['MES']=df_cuauh_v2.FECHA.apply(lambda x: str(x).split('-')[1])

In [282]:
df_cuauh_v2.head()

,FECHA,HORA,HGM,AÑO,MES
105192,2012-01-01,1,-99.0,2012,01
105193,2012-01-01,2,-99.0,2012,01
105194,2012-01-01,3,-99.0,2012,01
105195,2012-01-01,4,-99.0,2012,01
105196,2012-01-01,5,-99.0,2012,01


Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [284]:
df_cuauh_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2012,01,744
1,2012,02,696
2,2012,03,744
3,2012,04,720
4,2012,05,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

In [287]:
df_cuauh_v2[df_cuauh_v2.HGM == -99.0].groupby(["AÑO","MES"])['HGM'].value_counts().to_frame()

HGM
AÑO  MES HGM       
2012 01  -99.0  744
     02  -99.0  487
     03  -99.0   23
     04  -99.0   42
     05  -99.0  193
     06  -99.0  123
     07  -99.0   82
     08  -99.0  109
     09  -99.0   75
     10  -99.0   81
     11  -99.0  124
     12  -99.0   58
2013 01  -99.0   52
     02  -99.0   42
     03  -99.0   52
     04  -99.0   40
     05  -99.0   22
     06  -99.0   49
     07  -99.0   95
     08  -99.0  226
     09  -99.0  151
     10  -99.0  166
     11  -99.0   26
     12  -99.0   36
2014 01  -99.0   10
     02  -99.0   39
     03  -99.0   20
     04  -99.0   11
     05  -99.0  130
     06  -99.0  157
...             ...
2017 05  -99.0    7
     06  -99.0  126
     07  -99.0  355
     08  -99.0   77
     09  -99.0  180
     10  -99.0   65
     11  -99.0   27
     12  -99.0   22
2018 01  -99.0   13
     02  -99.0  552
     03  -99.0  225
     04  -99.0   19
     05  -99.0   28
     06  -99.0   35
     07  -99.0   26
     08  -99.0   26
     09  -99.0  496
     10  -99.0   87
     11  -99.0   13
     12  -99.0   14
2019 01  -99.0   32
     02  -99.0   34
     03  -99.0   37
     04  -99.0   11
     05  -99.0   29
     06  -99.0   24
     07  -99.0   25
     08  -99.0  495
     09  -99.0  254
     10  -99.0  372

[94 rows x 1 columns]

In [138]:
df_cuauh_v2.HGM.value_counts().head() # 11mil putos -99.0

-99.0    11202
 26.0     1125
 31.0     1112
 27.0     1106
 30.0     1106
Name: HGM, dtype: int64

Agregando columna AÑO y MES para hacer un groupby posterior, y así, encontrar los valores -99.0 para cada mes

In [280]:
df_cuauh_v2['AÑO']=df_cuauh_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_cuauh_v2['MES']=df_cuauh_v2.FECHA.apply(lambda x: str(x).split('-')[1])

In [282]:
df_cuauh_v2.head()

,FECHA,HORA,HGM,AÑO,MES
105192,2012-01-01,1,-99.0,2012,01
105193,2012-01-01,2,-99.0,2012,01
105194,2012-01-01,3,-99.0,2012,01
105195,2012-01-01,4,-99.0,2012,01
105196,2012-01-01,5,-99.0,2012,01


Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [284]:
df_cuauh_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2012,01,744
1,2012,02,696
2,2012,03,744
3,2012,04,720
4,2012,05,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

In [287]:
df_cuauh_v2[df_cuauh_v2.HGM == -99.0].groupby(["AÑO","MES"])['HGM'].value_counts().to_frame()

HGM
AÑO  MES HGM       
2012 01  -99.0  744
     02  -99.0  487
     03  -99.0   23
     04  -99.0   42
     05  -99.0  193
     06  -99.0  123
     07  -99.0   82
     08  -99.0  109
     09  -99.0   75
     10  -99.0   81
     11  -99.0  124
     12  -99.0   58
2013 01  -99.0   52
     02  -99.0   42
     03  -99.0   52
     04  -99.0   40
     05  -99.0   22
     06  -99.0   49
     07  -99.0   95
     08  -99.0  226
     09  -99.0  151
     10  -99.0  166
     11  -99.0   26
     12  -99.0   36
2014 01  -99.0   10
     02  -99.0   39
     03  -99.0   20
     04  -99.0   11
     05  -99.0  130
     06  -99.0  157
...             ...
2017 05  -99.0    7
     06  -99.0  126
     07  -99.0  355
     08  -99.0   77
     09  -99.0  180
     10  -99.0   65
     11  -99.0   27
     12  -99.0   22
2018 01  -99.0   13
     02  -99.0  552
     03  -99.0  225
     04  -99.0   19
     05  -99.0   28
     06  -99.0   35
     07  -99.0   26
     08  -99.0   26
     09  -99.0  496
     10  -99.0   87
     11  -99.0   13
     12  -99.0   14
2019 01  -99.0   32
     02  -99.0   34
     03  -99.0   37
     04  -99.0   11
     05  -99.0   29
     06  -99.0   24
     07  -99.0   25
     08  -99.0  495
     09  -99.0  254
     10  -99.0  372

[94 rows x 1 columns]

## Limpieza para Miguel Hidalgo

### Encontrar NaN's y -99.0

In [129]:
nulls_miguel = {}
for i in range(2000,2020):
    nulls_miguel[i]=(df_miguel[(df_miguel.FECHA >= '{}-01-01' .format(i)) & (df_miguel.FECHA <= '{}-12-31' .format(i))].isnull().sum()[2])

Las mediciones de esta estación empiezan a partir del 2015

In [130]:
nulls_miguel #Datos a partir del 2015

{2000: 8784,
 2001: 8760,
 2002: 8760,
 2003: 8760,
 2004: 8784,
 2005: 8760,
 2006: 8760,
 2007: 8760,
 2008: 8784,
 2009: 8760,
 2010: 8760,
 2011: 8760,
 2012: 8784,
 2013: 8760,
 2014: 8760,
 2015: 0,
 2016: 0,
 2017: 0,
 2018: 0,
 2019: 0}

Recortar el dataframe

In [139]:
df_miguel_v2=df_miguel[df_miguel.FECHA >= '2015-01-01']
df_miguel_v2.shape

(42360, 3)

In [140]:
df_miguel_v2.MGH.value_counts().head() # 6mil -99.0

-99.0    6048
 25.0     932
 27.0     896
 26.0     887
 23.0     874
Name: MGH, dtype: int64

Agregando columna AÑO y MES para hacer un groupby posterior, y así, encontrar los valores -99.0 para cada mes

In [ ]:
df_miguel_v2['AÑO']=df_miguel_v2.FECHA.apply(lambda x: str(x).split('-')[0])
df_miguel_v2['MES']=df_miguel_v2.FECHA.apply(lambda x: str(x).split('-')[1])

In [ ]:
df_miguel_v2.head()

Conteo de las mediciones de cada hora de cada día, de cada mes... Estas cantidades son los 'totales' para saber cual es el porcentaje de '-99.0' por cada mes

In [279]:
df_miguel_v2.groupby(["AÑO","MES"]).size().to_frame(name = 'count').reset_index().head()
#count = 744 cuando el mes tiene 31 días
#count = 672 cuando es febrero y ps 28 o 29 días...
#count = 720 cuando el mes tiene 30 días

,AÑO,MES,count
0,2015,01,744
1,2015,02,672
2,2015,03,744
3,2015,04,720
4,2015,05,744


Aca vemos cuántos '-99.0' hay para cada año-mes. Con esto podemos darnos una idea del impacto que tendría el funcionamiento del  modelo si es que decidimos eliminar esos rows, o reemplazar esos valores por otro.

Por ejemplo, para los últimos 5 meses del año 2019, practicamente no hay ningúna medición

In [278]:
df_miguel_v2[df_miguel_v2.MGH == -99.0].groupby(["AÑO","MES"])['MGH'].value_counts().to_frame()

MGH
AÑO  MES MGH       
2015 01  -99.0    7
     02  -99.0    6
     03  -99.0   34
     04  -99.0   58
     05  -99.0   83
     06  -99.0   38
     07  -99.0   14
     08  -99.0   12
     09  -99.0   29
     10  -99.0   31
     11  -99.0    5
     12  -99.0  144
2016 01  -99.0    5
     02  -99.0   10
     03  -99.0   21
     04  -99.0    4
     05  -99.0   11
     06  -99.0   23
     07  -99.0    9
     08  -99.0   12
     09  -99.0    5
     10  -99.0    6
     11  -99.0   11
     12  -99.0    7
2017 01  -99.0    6
     02  -99.0    6
     03  -99.0   11
     04  -99.0  266
     05  -99.0  272
     06  -99.0   19
     07  -99.0   29
     08  -99.0   46
     09  -99.0   38
     10  -99.0   46
     11  -99.0   20
     12  -99.0   33
2018 01  -99.0   10
     02  -99.0   32
     03  -99.0   11
     04  -99.0    4
     05  -99.0   13
     06  -99.0   19
     07  -99.0   15
     08  -99.0  286
     09  -99.0  153
     10  -99.0   25
     11  -99.0   63
     12  -99.0   18
2019 01  -99.0  128
     02  -99.0  207
     03  -99.0   26
     04  -99.0   40
     05  -99.0   26
     06  -99.0  643
     07  -99.0  744
     08  -99.0  744
     09  -99.0  720
     10  -99.0  744